# Fetch New data

Import necessary modules:

In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import requests
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import AllChem

## Data from DrugBank

Load the xml file downloaded from DrugBank.

In [ ]:
# Parse the DrugBank XML file
tree = ET.parse('data/full database.xml')
root = tree.getroot()

Extract data from xml file. Drug-target, drug-enzyme and drug-transporter were used.

In [ ]:
drug_target = []

# Iterate through drugs and extract target information
for drug in root.findall('{http://www.drugbank.ca}drug'):
    drug_name = drug.find('{http://www.drugbank.ca}name').text
    targets = drug.find('{http://www.drugbank.ca}targets')
    if targets is not None:
        for target in targets.findall('{http://www.drugbank.ca}target'):
            target_name = target.find('{http://www.drugbank.ca}name').text
            # print(f"Drug: {drug_name}, Target: {target_name}")
            drug_target.append(np.array([drug_name.upper(), target_name]))

drug_target = pd.DataFrame(np.vstack(drug_target))
drug_target

In [ ]:
# drug_enzyme = []

# # Iterate through drugs and extract target information
# for drug in root.findall('{http://www.drugbank.ca}drug'):
#     drug_name = drug.find('{http://www.drugbank.ca}name').text
#     enzymes = drug.find('{http://www.drugbank.ca}enzymes')
#     if enzymes is not None:
#         for enzyme in enzymes.findall('{http://www.drugbank.ca}enzyme'):
#             enzyme_name = enzyme.find('{http://www.drugbank.ca}name').text
#             # print(f"Drug: {drug_name}, Enzyme: {enzyme_name}")
#             drug_enzyme.append(np.array([drug_name.upper(), enzyme_name]))

# drug_enzyme = pd.DataFrame(np.vstack(drug_enzyme))
# drug_enzyme

In [ ]:
drug_enzyme = []

# Iterate through drugs and extract target information
for drug in root.findall('{http://www.drugbank.ca}drug'):
    drug_name = drug.find('{http://www.drugbank.ca}name').text
    enzymes = drug.find('{http://www.drugbank.ca}enzymes')
    if enzymes is not None:
        for enzyme in enzymes.findall('{http://www.drugbank.ca}enzyme'):
            enzyme_name = enzyme.find('{http://www.drugbank.ca}name').text
            # print(f"Drug: {drug_name}, Enzyme: {enzyme_name}")
            drug_enzyme.append(np.array([drug_name.upper(), enzyme_name]))

drug_enzyme = pd.DataFrame(np.vstack(drug_enzyme))
drug_enzyme

In [ ]:
drug_transporter = []

# Iterate through drugs and extract target information
for drug in root.findall('{http://www.drugbank.ca}drug'):
    drug_name = drug.find('{http://www.drugbank.ca}name').text
    transporters = drug.find('{http://www.drugbank.ca}transporters')
    if transporters is not None:
        for transporter in transporters.findall('{http://www.drugbank.ca}transporter'):
            transporter_name = transporter.find('{http://www.drugbank.ca}name').text
            # print(f"Drug: {drug_name}, Enzyme: {enzyme_name}")
            drug_transporter.append(np.array([drug_name.upper(), transporter_name]))

drug_transporter = pd.DataFrame(np.vstack(drug_transporter))
drug_transporter

In [ ]:
# drug_carrier = []

# # Iterate through drugs and extract target information
# for drug in root.findall('{http://www.drugbank.ca}drug'):
#     drug_name = drug.find('{http://www.drugbank.ca}name').text
#     carriers = drug.find('{http://www.drugbank.ca}carriers')
#     if carriers is not None:
#         for carrier in carriers.findall('{http://www.drugbank.ca}carrier'):
#             carrier_name = carrier.find('{http://www.drugbank.ca}name').text
#             drug_carrier.append(np.array([drug_name, carrier_name]))

# drug_carrier = pd.DataFrame(np.vstack(drug_carrier))
# drug_carrier

## ADR data

Preprocess the ADR data. Upper case of the drug names and ADR names were used to avoid miss-match and repetition. Only the drug-ADR pairs were used. A druglist that contain all drugs from SIDER and OFFSIDES was constructed for fetching drug-pathway and chemical structure fingerprint.

In [2]:
OFFSIDES=pd.read_csv("data/OFFSIDES.csv",sep = ",",header=0,index_col=0)
OFFSIDESdrug = np.unique(OFFSIDES["drug_concept_name"])
OFFSIDESdrug_upper = [drug_name.upper() for drug_name in OFFSIDESdrug]
df = pd.read_csv("data/drug_se.tsv",sep = "\t")
# drug_id = df["drugbank_id"] # put col of df in var
SIDERdrug = np.unique(df["1_x"])
SIDERdrug_upper = [drug_name.upper() for drug_name in SIDERdrug]
Alldrugs = np.unique(np.hstack([SIDERdrug_upper, OFFSIDESdrug_upper]))

<ipython-input-2-28c8982c3d5b>:1: DtypeWarning: Columns (0,2,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  OFFSIDES=pd.read_csv("data/OFFSIDES.csv",sep = ",",header=0,index_col=0)


## Drug-Pathway from KEGG

In [3]:
def search_drug(drug_name):
    url = f"http://rest.kegg.jp/find/drug/{drug_name}"
    response = requests.get(url)
    if response.status_code == 200:
        results = response.text.split('\n')
        return results
    else:
        print(f"Error: Failed to fetch data from KEGG API. Status code: {response.status_code}")
        return None

def get_drug_pathways(drug_id):
    url = f"http://rest.kegg.jp/link/pathway/{drug_id}"
    response = requests.get(url)
    if response.status_code == 200:
        pathways = []
        lines = response.text.split('\n')
        for line in lines:
            if line.strip():
                pathway_id = line.split('\t')[1].replace('path:', '')
                pathways.append(pathway_id)
        return pathways
    else:
        print(f"Error: Failed to fetch data from KEGG API. Status code: {response.status_code}")
        return None




drug_pathway = []
# Iterate through drugs and extract target information
i = 0
for drug in Alldrugs:
    i += 1
    drug_name = drug
    results = search_drug(drug_name)
    if results:
        # Extract the drug ID from the first result
        drug_id = results[0].split('\t')[0].replace('dr:', '')
        # Get pathways associated with the drug using its ID
        pathways = get_drug_pathways(drug_id)
        if pathways:
            for pathway in pathways:
               drug_pathway.append(np.array([drug_name, pathway]))
        else:
            print(f"No pathways found for drug{i}: {drug_name}.")
    else:
        print(f"No results found for drug{i}: {drug_name}.")

drug_pathway = pd.DataFrame(np.vstack(drug_pathway))
drug_pathway

Error: Failed to fetch data from KEGG API. Status code: 400
No pathways found for drug1: 1,25(OH)2D3.
Error: Failed to fetch data from KEGG API. Status code: 400
No pathways found for drug2: 17-ALPHA-HYDROXYPROGESTERONE.
Error: Failed to fetch data from KEGG API. Status code: 400
No pathways found for drug3: 17-HYDROXYPROGESTERONE.
Error: Failed to fetch data from KEGG API. Status code: 400
No pathways found for drug4: 18F-FDG.
Error: Failed to fetch data from KEGG API. Status code: 400
No pathways found for drug5: 18F-FLUTEMETAMOL.
Error: Failed to fetch data from KEGG API. Status code: 400
No pathways found for drug6: 2-HYDROXYETHYL METHACRYLATE.
Error: Failed to fetch data from KEGG API. Status code: 400
No pathways found for drug7: 2-HYDROXYSUCCINALDEHYDE.
Error: Failed to fetch data from KEGG API. Status code: 400
No pathways found for drug8: 2-MERCAPTOETHANESULFONIC ACID.
Error: Failed to fetch data from KEGG API. Status code: 400
No pathways found for drug9: 4-AP.
Error: Failed 

,0,1
0,5-FU,map00983
1,5-FU,map07041
2,6-MERCAPTOPURINE,map07041
3,ABACAVIR,map07044
4,ABACAVIR,map07053
...,...,...
1524,ZOTEPINE,map07028
1525,ZUCLOPENTHIXOL,map07028
1526,ZYPREXA,map07117
1527,ZYPREXA,map07211


## Chemical Structure Fingerprint

In [7]:
def get_compound_info(compound_name):
    # Base URL for PubChem Compound search
    base_url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound'

    # Construct the URL for compound search
    search_url = f'{base_url}/name/{compound_name}/JSON'

    # Send a GET request to retrieve compound information
    response = requests.get(search_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract compound information from the response JSON
        compound_info = response.json()

        # Check if the compound was found
        if 'PC_Compounds' in compound_info:
            # Extract SMILES string from the compound info
            
            props = compound_info['PC_Compounds'][0]['props']
            smiles_idx = np.where([((props[i]["urn"]['label'] == 'SMILES') and (props[i]["urn"]['name'] == 'Canonical')) for i in range(len(props))])[0][0]
            smiles = props[smiles_idx]['value']['sval']
            return smiles
        else:
            print(f"Compound '{compound_name}' not found in PubChem.")
            return None
    else:
        print(f"Failed to retrieve compound information. Status code: {response.status_code}")
        return None

drug_chemsfp = []
drugname_chemsfp = []
# Iterate through drugs and extract target information
i = 0
for drug in Alldrugs:
    i += 1
    drug_name = drug
    smiles = get_compound_info(drug_name)
    if smiles:
        # Convert SMILES to RDKit molecule object
        mol = Chem.MolFromSmiles(smiles)
        # Generate Morgan fingerprint (circular fingerprint)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
        # Convert RDKit fingerprint to binary string representation
        fingerprint_str = fp.ToBitString()
        fingerprint = np.array(list(fp.ToBitString()), dtype=int)
        # print(len(fingerprint))
        print(f"Chemical structure fingerprint for {i}. {drug_name}: {fingerprint_str}")
        drugname_chemsfp.append(drug_name)
        drug_chemsfp.append(np.array([fingerprint]))
# drugname_chemsfp = np.array(np.vstack(drugname_chemsfp))
# drug_chemsfp = pd.DataFrame(np.vstack(drug_chemsfp), index=drugname_chemsfp)
# drug_chemsfp
a = np.hstack(drugname_chemsfp)
drug_chemsfp = pd.DataFrame(np.vstack(drug_chemsfp), index=a)
drug_chemsfp

Failed to retrieve compound information. Status code: 404
Chemical structure fingerprint for 2. 17-ALPHA-HYDROXYPROGESTERONE: 0000000100000000100000000000000001001000000000000000000000000000000000010000000000001000100000000000000000000000000000000000000000100000001000000000000000000000000000000000000000000010000000000001000000000000000000000000000000000000000000000010000000100000000000100000000000000000000000000000000000000100000000000010000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000001000100000000000000000000000000000000000000000000000100000000000000000010000000000000000000000001001000000000001000000000100000000000000000000000001000000000000000000000000100000000000000000000000000000000000000000100000000000000000000001000010000000000000000000000000000000000000000000000000000000000000001000000000000000000000001000000000000000000000000000000000000000000000000000100000000000000001000000000000000000000000000000000100000001000000000010000000000010

[13:49:40] WARNING: not removing hydrogen atom without neighbors


Chemical structure fingerprint for 1095. FERROUS ASPARTO GLYCINATE: 01000000000000000000000000010000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000010000000000100000000010000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000100000000000000000000100000000000000000000000000000000000000000001000000000000000000000000000010000000000000000000000000000000000000000110000000000000000000010000000000000000010000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000

[13:50:55] WARNING: not removing hydrogen atom without neighbors
[13:50:55] WARNING: not removing hydrogen atom without neighbors


Chemical structure fingerprint for 1200. GADOBENATE: 01000000001000000000000000000000000000000010000000000000000000001000000000000000100000000000000000000000000000000010001000000000000000000000000001000000000000000000001010000001000000010000000000000000001000000000000000000000000000000001000000000000000100000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000100100000000000000000000000000000000000000000000000001000000000000000001000000000000000000000000000000100000000000000000000000000100000000000000000001000000000010000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000100000000000000000001000000000010001000100000000001000000000000000000000000000000000000000100000000000000000000010000000010000000000000000000000000000000010000000000000000000000000000000100000000000100000000000000000000000000000000010000000000000000000

...

Chemical structure fingerprint for 1210. GADOXETATE: 01000000000000000000000000000000010000000000000000000000000000000000010000000001100000000000000000000010001000000000000000000000000000000000000001000000000000000000001010000000000000010000000000000000001000000000001000000000000000000000010000000000000100000000000000001000000000000000000000000010000000000000001000000000001000000000000000000000000000000000100000000000000000000000000000100000000000000000000000000000000000000000100000000000000000000001000000000000000000000000000010000000000000000000000000000000000100000000000000000000000000000010000000000010000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000100000000000000000000000000000000000000000000100000000000000000001001000000010001000000000000000000000000000000000000000000000000000000100000000000000000000010000000000000000000000000000100000000000010000000100000000000000000000000100000000000100000000000000000000000000000000010000000000000000000

[13:51:13] WARNING: not removing hydrogen atom without neighbors
[13:51:13] WARNING: not removing hydrogen atom without neighbors


Chemical structure fingerprint for 1224. GD-EOB-DTPA: 0100000000000000000000000000000001000000000000000000000000000000000001000000000110000000000000000000001000100000000000000000000000000000000000000100000000000000000000101000000000000001000000000000000000100000000000100000000000000000000001000000000000010000000000000000100000000000000000000000001000000000000000100000000000100000000000000000000000000000000010000000000000000000000000000010000000000000000000000000000000000000000010000000000000000000000100000000000000000000000000001000000000000000000000000000000000010000000000000000000000000000001000000000001000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000010000000000000000000000000000000000000000000010000000000000000000100100000001000100000000000000000000000000000000000000000000000000000010000000000000000000001000000000000000000000000000010000000000001000000010000000000000000000000010000000000010000000000000000000000000000000001000000000000000000

[13:56:04] WARNING: not removing hydrogen atom without neighbors


Chemical structure fingerprint for 1626. LITHIUM ASPARTATE: 0100000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000001000000000000000000001000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000010000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000010000000000000000000010000000000000000000000000000000000000000000100000000000000000000000000001000000000000000000000000000000000000000011000000000000000000001000000000000000001000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000